# Importing everything 
Important: run this everytime!

In [ ]:
#(ignore) Importing everything
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, BaggingClassifier
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


import numpy as np

# EDA
Mencakup proses analisis untuk mendapatkan ide feature engineering, processing, dan insight yang didapatkan dari analisis data.

## Mari baca sekilas data yang kita punya!
Kita akan import datasets "training" untuk mendapat gambaran awal mengenai dataset kita

In [ ]:
# Load train.csv
kantor_file_path = pd.read_csv('../input/training/Train.csv')

# Menampilkan data secara umum
kantor_file_path.describe()


In [ ]:
# Menampilkan data bagian awal
kantor_file_path.head()

Dengan melihat dataset sekilas, kita dapat mengambil beberapa kesimpulan seperti:
1. Beberapa row masih mengandung data kosong (NaN), hal ini akan mempengaruhi proses prediksi nanti. Baiknya kita mengisi data kosong ini dengan value yang tidak merubah keseluruhan data secara signifikan
2. Beberapa data masih berupa text, kita harus mengubah text data ini menjadi suatu bilangan agar mudah dibaca oleh model kita.
3. Terdapat banyak sekali features, kita akan berusaha untuk mencari mana features yang lebih berperan dalam proses prediksi.

# Metode
Mencakup langkah preprocessing dan feature engineering.

## Random Sampling untuk mengisi data kosong (NaN)
Dari proses EDA, kita mengerti bahwa terdapat beberapa data kosong yang akan mengganggu proses prediksi kita. Kali ini, kita akan menggunakan metode "random sampling" untuk mengisi data kosong tadi.

Metode Random sampling untuk mengisi data kosong (NaN) adalah teknik di mana nilai-nilai yang hilang dalam dataset diisi menggunakan nilai-nilai yang diambil secara acak dari nilai yang sudah ada dalam kolom yang sama. Alhasil, dataset secara keseluruhan tidak akan berubah terlalu signifikan (maintain itself)

In [ ]:
# Membuat salinan DataFrame untuk manipulasi
kantor_data = kantor_file_path.copy()

# Kolom yang akan diisi dengan random sampling
# Kolom yang akan diisi dengan random sampling
columns_to_impute = [
    'Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 
    'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 
    'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 
    'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 
    'StandardHours', 'StockOptionLevel', 'WorkLifeBalance', 'YearsWithCurrManager',
    'BusinessTravel', 'Department', 'EducationField', 'Gender', 
    'OverTime', 'JobRole', 'MaritalStatus', 'OverTime'
]


# Mengisi nilai kosong dengan random sampling
for column in columns_to_impute:
    # Mendapatkan semua nilai non-NaN dari kolom tersebut
    non_nan_values = kantor_data[column].dropna().values  # Ubah ke array dengan .values
    
    # Fungsi untuk mengisi nilai kosong dengan random sampling
    def impute_random_value():
        return np.random.choice(non_nan_values)  # Mengambil sampel acak dari non-NaN
    
    # Terapkan imputasi random sampling ke kolom yang bersangkutan
    kantor_data[column] = kantor_data[column].apply(lambda x: impute_random_value() if pd.isna(x) else x)

## One Hot Encoding untuk mengubah data kategorikal menjadi angka biner
Selanjutnya, kita harus mengubah data kategorikal (seperti department, jobrole, dll) menjadi data angka yang bisa dibaca oleh model. Kita akan menggunakan one hot encoding untuk menyelesaikannya.

One hot encoding adalah metode yang digunakan untuk mengubah data kategorikal menjadi representasi biner agar dapat digunakan dalam algoritma pembelajaran mesin, yang umumnya hanya bekerja dengan data numerik. Proses ini mengonversi setiap kategori unik dalam variabel kategorikal menjadi kolom biner(0 atau 1).

In [ ]:
# --- Kolom yang ingin digunakan ---
columns_to_use = [
  'Age', 'EnvironmentSatisfaction', 'NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 
'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'OverTime', 'MaritalStatus'
]

# --- One-Hot Encoding untuk fitur kategorikal ---
ohe = OneHotEncoder(drop='first', sparse=False)
X_encoded = pd.DataFrame(
    ohe.fit_transform(
        kantor_data[['BusinessTravel', 'Department', 'EducationField', 'Gender',
                     'OverTime', 'JobRole', 'MaritalStatus']]
    )
)

# Memberikan nama kolom baru setelah One-Hot Encoding
X_encoded.columns = ohe.get_feature_names_out()

# Gabungkan hasil OHE dengan data numerik lainnya
kantor_data_final = pd.concat([kantor_data.drop(columns=['id','Attrition','EmployeeCount',
                                                         'Over18','StandardHours','BusinessTravel', 
                                                         'Department', 'EducationField', 'Gender', 
                                                         'OverTime', 'JobRole', 
                                                         'MaritalStatus']), X_encoded], axis=1)

In [ ]:
#Menampilkan beberapa baris hasil akhir
kantor_data_final.head()

Dengan begini, kita berhasil mengisi semua data kosong dan mengubah semua data kategorikal menjadi angka biner.

## Menentukan features yang akan dipakai dan target prediksi
Untuk sementara, kita akan menggunakan seluruh features yang ada. Disamping itu, kita tahu bahwa variabel yang kita cari adalah "Attritions"

In [ ]:
#assign features (variabel kolom) yang akan kita pakai dalam algoritma prediksi
kantor_features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Research & Development', 'Department_Sales',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'Gender_Male', 'OverTime_Yes',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'MaritalStatus_Married', 'MaritalStatus_Single']

In [ ]:
#assign X sebagai variabel (features) yg digunakan untuk memprediksi
X = kantor_data_final[kantor_features]

In [ ]:
#assign y sebagai variabel yg ingin diprediksi (atrisi)
y = kantor_data.Attrition

# **Evaluasi**
Mencakup modelling, evaluasi, dan analisis hasil prediksi.

## Modeling dengan RandomForestClassifier
Sekarang kita akan mulai tahap modeling, tepatnya kita akan menggunakan estimator "RandomForestClassifier" karena data prediksi yang diinginkan adalah value "true" atau "false".

Random Forest Classifier adalah algoritma pembelajaran mesin berbasis ensemble yang digunakan untuk tugas klasifikasi. Algoritma ini bekerja dengan membangun banyak decision trees selama pelatihan, dan kemudian menggabungkan hasilnya (melalui voting untuk klasifikasi) untuk menghasilkan prediksi yang lebih akurat dan stabil.



In [ ]:
#ini kalo treeclassifier
from sklearn.ensemble import RandomForestClassifier

#setup random seed. Why? idk. Biar sama trs kali?
np.random.seed(42)

#define "Kantor_model" as model (algoritma) yang kita pake
#Specify a number for random_state to ensure same results each run
kantor_model = RandomForestClassifier(n_estimators=100)

#melatih model dengan X, lalu si mesin bakal nyari pola agar hasilnya jadi y
kantor_model.fit(X, y)

## Ngecek f1 score dr model yg udah kita latih¶
"predicted_attritions" kita isi dengan hasil prediksi "kantor_model" dengan input variabel (features) X. Lalu aku mencari f1 scores dengan membandingkan y (hasil sebenarnya) dengan "predicted_attritions (hasil prediksiku)

Ideally ini nilainya 1.0

In [ ]:
#ngecek f1 score dr model yg udah kita latih
from sklearn.metrics import f1_score
predicted_attritions = kantor_model.predict(X)
f1_score(y, predicted_attritions)

## Nilai perfect doesn't mean modelnya perfect¶
algoritma model yang kita pilih adalah "RandomForestClassifier" dgn n_estimators=100. Idealnya, algoritma model yg sempurna adalah model yang kalau kita ngetrain pake dataset apapun HARUSNYA hasil prediksinya bener terus. Nah, kita mau ngecek nih kalo model kita cmn dilatih dengan setengah dr dataset awal kira-kira bisa menghasilkan prediksi yang bener engga.

Kita pake "train_test_split" untuk membelah data train jadi 2 bagian, anggep aja kalo datanya ada 100 aku bagi jadi 50 pasang x dan y.

yg satu namanya train_x (variabel latih) dan train_y (prediksi yang diinginkan dr latihan)
yg satu lagi namanya val_x (variabel validasi) dan val_y (prediksi validasi)
intinya, kita buang model sebelumnya, sekarang kita latih modelnya pake dataset baru (melatih model dengan train_X, lalu si mesin akan ,mencari pola agar hasilnya jadi train_y)

nah setelah polanya tersimpan, sekarang kita suruh prediksi dengan input "val_X", nanti hasil prediksinya kita masukin ke "val_predictions"

"val_predictions" ini kita bandingkan dengan "val_y" untuk mengecek f1 scorenya. KALAU polanya sempurna, harusnya f1 scorenya 1.0 karena ideally hasil prediksi kita sama seperti "val_y"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# ini ngesplit datanya, dikasih randomstate biar kebelahnya di posisi yang sama terus
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# define "Kantor_model" as model (algoritma) yang kita pake
kantor_model = RandomForestClassifier(max_depth=50, max_features=None, n_estimators=100, random_state=1)

# melatih model dengan train_X, lalu si mesin bakal nyari pola agar hasilnya jadi train_y
kantor_model.fit(train_X, train_y)

#check akurat
print(kantor_model.score(val_X, val_y))

# ngecek f1 score dengan mbandingin val_y (prediksi valid)
# dgn val_predictions (hasil prediksi dgn input val_X)
val_predictions = kantor_model.predict(val_X)
print(f1_score(val_y, val_predictions))

## Meningkatkan akurasi dan presisi model kita
Dari sini, kita paham bahwa model kita tidak sempurna. Random Forest *by itself* tidaklah cukup akurat untuk menghasilkan prediksi yang dapat diandalkan. Untuk meningkatkan tingkat akurasi model kita, ada beberapa hal yang dapat dilakukan:

1. Mencari Feature yang penting
2. Menggunakan model lain yang lebih cocok
3. Tuning model

## Mencari Feature yang Penting
*not every features are created equal*

Dalam melakukan prediksi, model akan menggunakan features sebagai pola dalam prosesnya. Namun, tidak semua features memiliki tingkat "pengaruh" yang sama. Kita akan mencari mana features yang paling berpengaruh dan mana yang kurang berpengaruh dengan metode SFS (Sequential Feature Selection)

Metode Sequential Feature Selection (SFS) adalah salah satu teknik dalam feature selection yang digunakan untuk memilih subset fitur yang paling relevan dari data, dengan tujuan meningkatkan kinerja model dan mengurangi kompleksitas. SFS melibatkan pendekatan iteratif di mana fitur-fitur dipilih secara berurutan berdasarkan kontribusi mereka terhadap kinerja model. Terdapat dua metode SFS yaitu forward SFS (dimulai tanpa apapun, satu per satu menambah feature) dan backward SFS (dimulai dengan segalanya, satu per satu mengurangi feature)

Tepatnya, kita akan menggunakan metode Forward SFS.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Misalkan kita sudah memiliki data dan target
# X -> data fitur
# y -> target

# Membagi data ke dalam training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Pilih RandomForest sebagai estimator
rf_model = RandomForestClassifier(random_state=42)

# Buat Sequential Feature Selector (SFS) dengan mlxtend (forward selection)
sfs = SFS(rf_model,
          k_features=10,  # Memilih 10 fitur terbaik
          forward=True,   # Forward selection
          floating=False, # Floating bisa diaktifkan untuk fleksibilitas tambahan
          scoring='f1',   # Gunakan F1 score untuk evaluasi
          cv=5,           # 5-fold cross-validation untuk stabilitas
          n_jobs=-1)      # Gunakan semua core CPU untuk efisiensi

# Fit SFS pada data training
sfs = sfs.fit(X_train, y_train)

# Fitur terbaik yang dipilih
selected_features = list(sfs.k_feature_names_)

# Transformasi data dengan fitur yang dipilih
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

# Latih ulang model dengan fitur yang dipilih
rf_model.fit(X_train_selected, y_train)

# Prediksi dan evaluasi model
y_pred = rf_model.predict(X_test_selected)
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

# Menampilkan fitur yang dipilih
print(f"Selected Features: {selected_features}")

Nah, sekarang kita akan assign ulang features yang digunakan dengan hasil dari metode Forward SFS tadi.

In [ ]:
# Load file CSV
kantor_file_path = pd.read_csv('../input/training/Train.csv')

# Membuat salinan DataFrame untuk manipulasi
kantor_data = kantor_file_path.copy()

# Kolom yang akan diisi dengan random sampling dari fitur yang dipilih oleh SFS
columns_to_impute = ['Age','NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
    'YearsWithCurrManager', 'Department', 'OverTime', 'MaritalStatus']
# Mengisi nilai kosong dengan random sampling
for column in columns_to_impute:
    # Mendapatkan semua nilai non-NaN dari kolom tersebut
    non_nan_values = kantor_data[column].dropna().values  # Ubah ke array dengan .values
    
    # Fungsi untuk mengisi nilai kosong dengan random sampling
    def impute_random_value():
        return np.random.choice(non_nan_values)  # Mengambil sampel acak dari non-NaN
    
    # Terapkan imputasi random sampling ke kolom yang bersangkutan
    kantor_data[column] = kantor_data[column].apply(lambda x: impute_random_value() if pd.isna(x) else x)

# --- Kolom yang ingin digunakan ---
columns_to_use = ['Age','NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
    'YearsWithCurrManager', 'Department', 'OverTime', 'MaritalStatus']

# --- One-Hot Encoding untuk fitur kategorikal ---
categorical_columns = ['Department', 'OverTime','MaritalStatus']

# Menggunakan One-Hot Encoder hanya pada kolom kategorikal
ohe = OneHotEncoder(drop='first', sparse=False)
X_encoded = pd.DataFrame(
    ohe.fit_transform(kantor_data[categorical_columns])
)

# Memberikan nama kolom baru setelah One-Hot Encoding
X_encoded.columns = ohe.get_feature_names_out(categorical_columns)

# Gabungkan hasil OHE dengan data numerik lainnya menjadi kesatuan data yang digunakan untuk pelatihan
kantor_data_final = pd.concat([kantor_data[columns_to_use].drop(columns=categorical_columns), X_encoded], axis=1)

kantor_data_final.head()

In [ ]:
#assign features (variabel kolom) yang akan kita pakai dalam algoritma prediksi setelah SFS
kantor_features = ['Age', 'NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
                   'YearsWithCurrManager', 'Department_Research & Development', 'OverTime_Yes', 'MaritalStatus_Single']

#assign X sebagai variabel (features) yg digunakan untuk memprediksi
X = kantor_data_final[kantor_features]

## Menggunakan atau menambah model yang digunakan
Untuk kasus kali ini, kita akan menggunakan lebih dari satu model dengan metode stacking.

Stacking (atau stacked generalization) adalah metode ensemble learning di mana beberapa model pembelajaran mesin (disebut base models atau level-1 models) digabungkan untuk meningkatkan kinerja prediksi.

Kita akan menggunakan estimator RandomForestClassifier dan GradientBoostingClassifier

In [ ]:
# Membagi data ke dalam training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model dasar (base models)
# Menggunakan RandomForest dan Support Vector Machine sebagai base models
base_models = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))  # SVC dengan probability=True untuk stacking
]

# Model meta (final estimator)
# Menggunakan Logistic Regression sebagai meta-model
meta_model = LogisticRegression()

# Stacking Classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Fit Stacking Classifier pada data training
stacking_clf.fit(X_train, y_train)

# Prediksi pada data testing
y_pred = stacking_clf.predict(X_test)

In [ ]:
# Evaluasi model dengan F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score Stacking Classifier: {f1}")

# Menampilkan hasil prediksi
print(f"Prediksi: {y_pred}")
val_predictions = stacking_clf.predict(X_test)
print(f1_score(y_test, val_predictions))

## Model akhir
Kita akan stay dengan model ini. Saatnya retrain model kita

In [ ]:
stacking_clf.fit(X, y)

# Mencari Prediksi test.csv
Kita akan mencoba untuk prediksi dengan input test_data.

## Membaca file test.csv, random sampling, OHE, dan assign features.

In [ ]:
# Load file CSV
kantortest_file_path = pd.read_csv('../input/testing/Test.csv')

# Membuat salinan DataFrame untuk manipulasi
kantortest_data = kantortest_file_path.copy()

# Kolom yang akan diisi dengan random sampling dari fitur yang dipilih oleh SFS
columns_to_impute = ['Age','NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
    'YearsWithCurrManager', 'Department', 'OverTime', 'MaritalStatus']

# Mengisi nilai kosong dengan random sampling
for column in columns_to_impute:
    # Mendapatkan semua nilai non-NaN dari kolom tersebut
    non_nan_values = kantortest_data[column].dropna().values  # Ubah ke array dengan .values
    
    # Fungsi untuk mengisi nilai kosong dengan random sampling
    def impute_random_value():
        return np.random.choice(non_nan_values)  # Mengambil sampel acak dari non-NaN
    
    # Terapkan imputasi random sampling ke kolom yang bersangkutan
    kantortest_data[column] = kantortest_data[column].apply(lambda x: impute_random_value() if pd.isna(x) else x)

#OHE PART    
# --- Kolom yang ingin digunakan ---
columns_to_use = ['Age','NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
    'YearsWithCurrManager', 'Department', 'OverTime', 'MaritalStatus']

# --- One-Hot Encoding untuk fitur kategorikal ---
categorical_columns = ['Department', 'OverTime','MaritalStatus']

# Menggunakan One-Hot Encoder hanya pada kolom kategorikal
ohe = OneHotEncoder(drop='first', sparse=False)
X_encoded = pd.DataFrame(
    ohe.fit_transform(kantortest_data[categorical_columns])
)

# Memberikan nama kolom baru setelah One-Hot Encoding
X_encoded.columns = ohe.get_feature_names_out(categorical_columns)

# Gabungkan hasil OHE dengan data numerik lainnya menjadi kesatuan data yang digunakan untuk pelatihan
kantortest_data_final = pd.concat([kantortest_data[columns_to_use].drop(columns=categorical_columns), X_encoded], axis=1)



In [ ]:
# Menampilkan beberapa baris hasil akhir
kantortest_data_final.head()

In [ ]:
#assign features (variabel kolom) yang akan kita pakai dalam algoritma prediksi
kantortest_features = ['Age', 'NumCompaniesWorked', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 
                   'YearsWithCurrManager', 'Department_Research & Development', 'OverTime_Yes', 'MaritalStatus_Single']

In [ ]:
#nge-assign variabel (features) test_data yg akan kita pake
X_test = kantortest_data_final[kantortest_features]

## Prediksi

In [ ]:
#memprediksi pake input "X_test" dengan model yg udah kita batesin jumlah cabangnya
predictions = stacking_clf.predict(X_test)

## Membuat file csv untuk di-submit

In [ ]:
#ini cuman buat nambahin kolom "Attrition" di file output csv-nya
kantortest_data['Attrition'] = predictions

In [ ]:
# Membuat DataFrame untuk hasil prediksi
index_values = range(len(predictions))  # Indeks dari 0 hingga panjang prediksi
results = pd.DataFrame({
    'id': index_values,
    'Attrition': ['TRUE' if pred >= 0.5 else 'FALSE' for pred in predictions]
})

# Menyimpan DataFrame ke file CSV
results.to_csv('predictions.csv', index=False)

In [ ]:
from IPython.display import FileLink

# Buat link untuk mendownload file
FileLink('predictions.csv')